In [2]:
import numpy as np
from matplotlib import pyplot as mpl, cm
import itertools
import pandas as pd
import csv

In [3]:
#Primer problema: Cancer de Mamas

M = 1 #Salida :  Maligno (T) , Benigno (F)
N = 10 #Entradas
P = 411 #instancias
p_train = 325
p_check = P - p_train

#los usamos para entrenar
entradas_train = np.random.normal(0, 0.5 , (p_train,N+1))
entradas_train[:,N] = 1     #umbral
tags_train = np.zeros((p_train,1))


#se usan despues para verificar
entradas_check  = np.random.normal(0, 0.5 , (p_check,N+1))
entradas_check[:,N] = 1    #umbral
tags_check = np.zeros((p_check,1))

dataPath = 'tp1_ej1_training.csv'
dataViewer = pd.read_csv(dataPath, names =['Z','a','b','c','d','e','f','g','h','i','j'])


with open(dataPath) as csvfile:
    dataCSV = csv.reader(csvfile, delimiter=',')
    i=0
    k=0
    for row in dataCSV:
        if i < p_train:
            
            if row[0] == 'M':
                tags_train[i][0] = 1
            else:
                tags_train[i][0] = -1

            j=0
            for col in range(1,len(row)):
                entradas_train[i][j] = row[col]
                j+=1
            i +=1
        else: 
            if row[0] == 'M':
                tags_check[k][0] = 1
            else:
                tags_check[k][0] = -1

            j=0
            for col in range(1,len(row)):
                entradas_check[k][j] = row[col]
                j+=1
            k+=1    
        
    

In [13]:
def bias_add( V):
    bias = -np.ones( (len(V),1) )
    return np.concatenate( (V,bias), axis=1)

def bias_sub( V):
    return V[:,:-1]

In [14]:
#Clase capa de la red
# Es una base, podemos agregarle otros parametros por ahi
class redNeuronal():
    
        
    
    def __init__(self, dim_capas, entradas, f_activ):
        
        self.dim_capas = dim_capas
        self.entradas = entradas
        self.f_activ = f_activ
        #dim_capas = [N,3,3,3,1]
        self.W = [] #primera pos que sea nula
        self.n = len(dim_capas)
        self.lr = 0.3   #learning rate
    
        #creo las matrices de Weights
        W.append(0) #primera capa no tiene wieghts asociados
        for k in range(1,n):
            W.append( np.random.normal( 0, 0.5, (dim_capas[i-1], dim_capas[i])) )
        
        
        
    def activation(X):
        Y = []

        #llenar las capas con sus dimensiones correspondientes
        Y0 = np.zeros( (1, dim_capas[0]+1) )
        Y0[:] = bias_add(X)
        Y.append(Y0)
        for i in range(1,n-1):
            #creo todas las capas del medio con sus dims correspondientes
            Yi = np.zeros((1, dim_capas[i] + 1))
            Yi[:] = bias_add( np.tanh( np.dot( Y[i-1],W[i] )))   #funcion de act tanh
            Y.append(Yi)
        Yn = np.zeros( (1, dim_capas[-1])) #capa de salida
        Yn[:] = np.tanh(np.dot(Y[-1],W[-1])) #ultima capa con ultimos Weights
        Y.append(Yn)
        
        return Y
        
    def backprop(Y, Z):
        dW = []
        dW.append(0) #el primero no tiene weights
        #creo los dW con las mismas dimensiones que sus respectivos W para no tener conflicto de cuentas
        for i in range(1,n):
            dWi = np.zeros_like(W[i])
            dW.append(dWi)
        
        D = [0] * n   #lista vacia para los D's
        
        En = Z-Y[-1]  #error de ultima capa
        dYn = 1-np.square(Y[-1])
        Dn = En*dYn
        D[-1] = Dn
        
        
        
        for k in range(n-1,1):
            dW[k] += lr * np.dot(Y[k-1].T, D[k])
            Ei = np.dot(D[k], W[k].T)
            dYi = 1-np.square(Y[k-1])
            Di = bias_sub(Ei * dYi)
            D[k-1] = Di
        
        return dW
            
        
        
        
        

In [ ]:
def layers(X,S):
    
    #capas
    Y0 = np.zeros( (1,S[0]+1) )
    Y1 = np.zeros( (1,S[1]+1) )
    Y2 = np.zeros( (1,S[2]+1) )
    Y3 = np.zeros( (1,S[3]) )
    
    W1 = np.random.normal( 0, 0.5, (S[0]+1, S[1]))
    W2 = np.random.normal( 0, 0.5, (S[1]+1, S[2]))
    W3 = np.random.normal( 0, 0.5, (S[2]+1, S[3]))
    
    
    #batch
    B = 1
    H = np.random.permutation(P)
    
    #learning rate y error por epoca
    lr = 0.3
    error = 0.0

    for i in range(len(H)):
        
        h = H[i]

        Xh = X[h:h+B]
        Zh = Z[h:h+B]
        
        #feed forward 
        #activacion tanh (sigmoidea)
        Y0[:] = bias_add(Xh)
        Y1[:] = bias_add( np.tanh(np.dot(Y0,W1)))
        Y2[:] = bias_add( np.tanh(np.dot(Y1,W2)))
        Y3[:] = np.tanh(np.dot(Y2,W3))

        dW1 = np.zeros_like(W1)
        dW2 = np.zeros_like(W2)
        dW3 = np.zeros_like(W3)
        
        #backpropagation
        
        E3 = Zh-Y3
        dY3 = 1-np.square(Y3)
        D3 = E3*dY3
        
        dW3 += lr * np.dot(Y2.T,D3)
        
        E2 = np.dot(D3, W3.T)
        dY2 = 1-np.square(Y2)
        D2 = bias_sub(E2*dY2)

        dW2 += lr * np.dot( Y1.T, D2)

        E1 = np.dot( D2, W2.T)
        dY1 = 1-np.square(Y1)
        D1 = bias_sub( E1*dY1)

        dW1 += lr * np.dot( Y0.T, D1)

        W1 += dW1
        W2 += dW2
        error += np.mean( np.sum( np.square(Zh-Y2),axis=1))
    
    #prueba con red entrenada
    
    
    Y0 = np.zeros( (P,S[0]+1) )
    Y1 = np.zeros( (P,S[1]+1) )
    Y2 = np.zeros( (P,S[2]+1) )
    Y3 = np.zeros( (P,S[3]) )
    
    Y0[:] = bias_add(X)
    Y1[:] = bias_add( np.tanh(np.dot(Y0,W1)))
    Y2[:] = bias_add(np.tanh(np.dot(Y1,W2)))
    Y3[:] = np.tanh(np.dot(Y2,W3))
    return Y3

In [13]:
#funciones de ACtivacion (Justo, vos las tenias piolas)